<a href="https://colab.research.google.com/github/Jaksta1/Uczenie_Maszynowe_2025/blob/main/Jakub_Kownacki_praca_domowa_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---------------------------------
# 1. Importowanie bibliotek
---------------------------------

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.animation as animation
from IPython.display import HTML
plt.rcParams['animation.embed_limit'] = 50 #zwiększenie limitu pamięci do stworzenia animacji
# Ustawienie ziarna dla powtarzalności
torch.manual_seed(42)
np.random.seed(42)

-------------------------------------------
#2. Ustawienia parametrów
-------------------------------------------

In [ ]:
# Parametry elipsy
focus1 = torch.tensor([-2.0, 0.0])  # Pierwsze ognisko
focus2 = torch.tensor([2.0, 0.0])   # Drugie ognisko
constant_sum = 6.0                  # Stała suma odległości

# Inicjalizacja losowych punktów
num_points = 100
points = torch.rand((num_points, 2)) * 10 - 5  # Rozkład jednostajny w [-5, 5]
points.requires_grad = True

------------------------------------------------
#3. Funkcje pomocnicze
------------------------------------------------

In [ ]:
# Funkcja obliczająca odległości
def compute_distances(points, focus1, focus2):
    dist1 = torch.norm(points - focus1, dim=1)
    dist2 = torch.norm(points - focus2, dim=1)
    return dist1, dist2

def l0_approx_sigmoid(epsilon, delta, alpha=50):
    """
    Aproksymacja normy L0 za pomocą funkcji sigmoidalnej.
    Parametry:
        epsilon: tensor wartości błędów
        delta: próg, poniżej którego błędy są uznawane za zerowe
        alpha: parametr kontrolujący nachylenie funkcji sigmoidalnej, większe wartości alpha sprawiają,
        że aproksymacja staje się bardziej "skokowa", zbliżając się do idealnej normy L0
        różniczkowalność: funkcja sigmoidalna jest różniczkowalna, zatem możemy jej użyć do
        obliczenia gradientu funkcji straty.
    Zwraca:
        aproksymowaną wartość normy L0
    """
    return 1 / (1 + torch.exp(-alpha * (torch.abs(epsilon) - delta)))

# Funkcja treningowa z zapisem trajektorii i straty
def train_with_animation(loss_type, num_epochs=1000, lr=0.1):
    points = torch.rand((num_points, 2)) * 10 - 5
    points.requires_grad = True
    optimizer = torch.optim.Adam([points], lr=lr)
    trajectories = []  # Lista do przechowywania położenia punktów w każdej epoce
    loss_history = []  # Lista do przechowywania wartości straty w każdej epoce

    for epoch in range(num_epochs):
        optimizer.zero_grad()
        dist1, dist2 = compute_distances(points, focus1, focus2)
        epsilon = dist1 + dist2 - constant_sum

        if loss_type == "l2":
            loss = torch.mean(epsilon ** 2)
        elif loss_type == "l1":
            loss = torch.mean(torch.abs(epsilon))
        elif loss_type == "linf":
            loss = torch.max(torch.abs(epsilon))
        elif loss_type == "l0":
            delta = 0.01
            loss = torch.mean(l0_approx_sigmoid(epsilon, delta))  # Użyj aproksymacji sigmoidalnej
        else:
            raise ValueError("Nieprawidłowy typ straty")

        loss.backward()
        optimizer.step()

        # Zapisz aktualne położenia punktów i stratę
        trajectories.append(points.detach().clone().numpy())
        loss_history.append(loss.item())

    return trajectories, loss_history

---------------------------------------------
#4. Trening dla straty L1
---------------------------------------------

In [ ]:
# Uruchom trening dla straty l1
trajectories_l1, loss_history_l1 = train_with_animation("l1", num_epochs=300)

# Tworzenie animacji z dwoma subplotami
fig_l1, (ax1_l1, ax2_l1) = plt.subplots(1, 2, figsize=(12, 6))

# Subplot 1: Punkty
ax1_l1.set_xlim(-6, 6)
ax1_l1.set_ylim(-6, 6)
scatter_l1 = ax1_l1.scatter([], [], label='Punkty')
ax1_l1.scatter([focus1[0], focus2[0]], [focus1[1], focus2[1]], color='red', marker='x', s=100, label='Ogniska')
ax1_l1.legend()
ax1_l1.grid()
ax1_l1.set_title('Ruch punktów dla straty L1')

# Subplot 2: Wykres straty
ax2_l1.set_xlim(0, len(loss_history_l1))
ax2_l1.set_ylim(0, max(loss_history_l1) * 1.1)
line_l1, = ax2_l1.plot([], [], color='blue')
ax2_l1.set_xlabel('Epoka')
ax2_l1.set_ylabel('Strata')
ax2_l1.set_title('Przebieg funkcji straty L1')
ax2_l1.grid()

def update_l1(frame):
    # Aktualizacja położenia punktów
    scatter_l1.set_offsets(trajectories_l1[frame])

    # Aktualizacja wykresu straty
    line_l1.set_data(range(frame + 1), loss_history_l1[:frame + 1])

    return scatter_l1, line_l1

ani_l1 = animation.FuncAnimation(fig_l1, update_l1, frames=len(trajectories_l1), interval=50, blit=True)
print("Ostatnia wartość funkcji błędu L1:", loss_history_l1[-1])

# Wyświetlenie animacji w notebooku
HTML(ani_l1.to_jshtml())


##Dlaczego strata $L^1$ nie zbiega do zera nawet po narysowaniu elipsy?

Norma L1 to $𝐿_{\text{ellipse}}^{(1)}=\frac{1}{𝑁}∑_{𝑖=1}^𝑁∣𝜖_𝑖∣$, czyli średnia wartość bezwzględna błędów.\
Powodem nie zbiegania straty L1 do 0 jest to, że w przeciwieństwie do normy L2 (błędu kwadratowego), która silnie karze nawet małe odchylenia (gradient rośnie z błędem), norma L1 ma stały gradient (±1 w zależności od znaku $𝜖_𝑖 $​
 ). To sprawia, że małe błędy są słabo penalizowane, więc punkty mogą być blisko elipsy, ale nie dokładnie na niej. W związku z tym optymalizacja może oscylować lub zwolnić, gdy błędy są małe, co prowadzi do ustabilizowania wartości straty na poziomie powyżej zera.\
**Wynik**: Strata nie zbiega do zera, bo L1 nie wymusza precyzyjnego dopasowania punktów do warunku.\
Wszystkie te wnioski są zgodne z powyższą animacją, na której widać, że dla odpowiednio dużej liczby iteracji błąd utrzymuje się na stałej wartości około 0.02 pomimo tego, że uzyskaliśmy elipsę.

----------------------------------------------
#5. Trening dla straty $L^{∞}$
-----------------------------------------------

In [ ]:
# Uruchom trening dla straty L_inf
trajectories_linf, loss_history_linf = train_with_animation("linf", num_epochs=800)

# Tworzenie animacji z dwoma subplotami
fig_linf, (ax1_linf, ax2_linf) = plt.subplots(1, 2, figsize=(12, 6))

# Subplot 1: Punkty
ax1_linf.set_xlim(-6, 6)
ax1_linf.set_ylim(-6, 6)
scatter_linf = ax1_linf.scatter([], [], label='Punkty')
ax1_linf.scatter([focus1[0], focus2[0]], [focus1[1], focus2[1]], color='red', marker='x', s=100, label='Ogniska')
ax1_linf.legend()
ax1_linf.grid()
ax1_linf.set_title('Ruch punktów dla straty L_inf')

# Subplot 2: Wykres straty
ax2_linf.set_xlim(0, len(loss_history_linf))
ax2_linf.set_ylim(0, max(loss_history_linf) * 1.1)
line_linf, = ax2_linf.plot([], [], color='blue')
ax2_linf.set_xlabel('Epoka')
ax2_linf.set_ylabel('Strata')
ax2_linf.set_title('Przebieg funkcji straty L_inf')
ax2_linf.grid()

def update_linf(frame):
    # Aktualizacja położenia punktów
    scatter_linf.set_offsets(trajectories_linf[frame])

    # Aktualizacja wykresu straty
    line_linf.set_data(range(frame + 1), loss_history_linf[:frame + 1])

    return scatter_linf, line_linf

ani_linf = animation.FuncAnimation(fig_linf, update_linf, frames=len(trajectories_linf), interval=50, blit=True)
print("Ostatnia wartość funkcji błędu L_inf:", loss_history_linf[-1])

# Wyświetlenie animacji w notebooku
HTML(ani_linf.to_jshtml())


##Dlaczego trening trwa długo i strata nie zbiega do zera?

Norma $L_{\text{inf}}$ to maksymalny błąd wśród wszystkich punktów.\
Zatem optymalizacja koncentruje się wyłącznie na punkcie z największym błędem, ignorując pozostałe, dopóki ten błąd nie zostanie zredukowany. To powoduje powolny postęp, bo tylko jeden punkt (lub kilka) jest poprawianych w każdej iteracji. Istnieje też ryzyko przełączanie się między punktami o największym błędzie, co prowadzi do nieefektywnego ruchu w przestrzeni, a brak równoczesnej poprawy wszystkich punktów utrudnia pełne dopasowanie do elipsy.\
**Wynik**: Trening jest wolny i często nie zbiega do zera, bo strategia "najgorszego przypadku" nie optymalizuje globalnie układu punktów.\
Na powyższej animacji ruchu punktów widać, że na początku udaje się uzyskać "zarys" elipsy, jednak w pewnym momencie algorytmowi ciężko wybrać punkty i kierunek ich przesunięcia taki, aby zmniejszyć błąd, co widać na animacji przebiegu funkcji straty.

-----------------------------------------------
#6. Trening dla straty L0
----------------------------------------------

In [ ]:
# Uruchom trening dla straty l0
trajectories_l0, loss_history_l0 = train_with_animation("l0", num_epochs=300)

# Tworzenie animacji z dwoma subplotami
fig_l0, (ax1_l0, ax2_l0) = plt.subplots(1, 2, figsize=(12, 6))

# Subplot 1: Punkty
ax1_l0.set_xlim(-6, 6)
ax1_l0.set_ylim(-6, 6)
scatter_l0 = ax1_l0.scatter([], [], label='Punkty')
ax1_l0.scatter([focus1[0], focus2[0]], [focus1[1], focus2[1]], color='red', marker='x', s=100, label='Ogniska')
ax1_l0.legend()
ax1_l0.grid()
ax1_l0.set_title('Ruch punktów dla straty L0')

# Subplot 2: Wykres straty
ax2_l0.set_xlim(0, len(loss_history_l0))
ax2_l0.set_ylim(0, max(loss_history_l0) * 1.1 if loss_history_l0 else 1)
line_l0, = ax2_l0.plot([], [], color='blue')
ax2_l0.set_xlabel('Epoka')
ax2_l0.set_ylabel('Strata')
ax2_l0.set_title('Przebieg funkcji straty L0')
ax2_l0.grid()

def update_l0(frame):
    # Aktualizacja położenia punktów
    scatter_l0.set_offsets(trajectories_l0[frame])

    # Aktualizacja wykresu straty
    line_l0.set_data(range(frame + 1), loss_history_l0[:frame + 1])

    return scatter_l0, line_l0

ani_l0 = animation.FuncAnimation(fig_l0, update_l0, frames=len(trajectories_l0), interval=50, blit=True)
print("Ostatnia wartość funkcji błędu L0:", loss_history_l0[-1])

# Wyświetlenie animacji w notebooku
HTML(ani_l0.to_jshtml())

##Dlaczego trening nie postępuje?
Norma L0 jest zdefiniowana jako $L^{(0)}_{\text{ellipse}}=\frac{1}{N}∑_{i=1}^N\mathbb{1}(ϵ_i\neq1)$.\
Norma L0 zlicza więc stosunek liczby punktów, dla których błąd $𝜖_𝑖$ nie jest równy zero do liczby wszystkich punktów.\
Trening nie postępuje, ponieważ indykator $\mathbb{1}_(ϵ_i\neq1)$ jest funkcją skokową – zmienia się z 0 na 1 w sposób dyskretny, nie jest to zatem funkcja różniczkowalna. W optymalizacji gradientowej potrzebujemy ciągłych gradientów, a norma L0 ich nie dostarcza. Nawet jeśli spróbujemy ją przybliżyć np. używając funkcji sigmoidalnej, gradienty są zerowe, co uniemożliwia skuteczne kierowanie punktami w stronę elipsy.\
**Wynik**: Trening nie postępuje, bo optymalizator nie otrzymuje użytecznych informacji o tym, jak dostosować pozycje punktów.\
Wnioski te są zgodne z animacją ruchu punktów, na której widać, że większość punktów jest nieruchoma, niektóre punkty nieznacznie się przemieszczają, jednak mimo to nie tworzą elipsy.\
Na wykresie straty widzimy, że strata jest prawie stale równa 1, co jest zgodne z brakiem zbliżania się do kształtu elipsy punktów przedstawionych w animacji.